$ \hat{H}_{tr} = \omega_r \hat{a}^\dagger \hat{a} + \hat{H}_t - i g (\hat{n}_t - n_g)(\hat{a} - \hat{a}^\dagger) $

In [ ]:

import scqubits as scq
import qutip as qt
import numpy as np


EC = 0.2
EJ = 15 # 0.22 * 110
E_osc = 6.8  # GHz, resonator frequency

tmon = scq.Transmon(
    EJ=EJ,      # Josephson energy
    EC=EC,      # Charging energy
    ng=0.3,     # Offset charge
    ncut=41,    # Charge basis cutoff
    truncated_dim=10  # Number of levels to keep
)


resonator = scq.Oscillator(E_osc=E_osc, truncated_dim=20)

hilbertspace = scq.HilbertSpace([tmon, resonator])

g = -0.120*1j #g = 120 MHz, calculate this from  capacitance matrix

hilbertspace.add_interaction(
    expr="g * n * a - g * n * adag",
    op1=("n", tmon.n_operator),
    op2=("adag", resonator.creation_operator),
    op3 = ("a", resonator.annihilation_operator),
    add_hc=False
)

hilbertspace.generate_lookup()
    
    # Extract eigenvalues
evals = hilbertspace["evals"][0]
diag_dressed_hamiltonian = (
    2 * np.pi * qt.Qobj(np.diag(evals), dims=[hilbertspace.subsystem_dims] * 2)
    )

total_truncation = 10
def truncate(operator: qt.Qobj, dimension: int) -> qt.Qobj:
        return qt.Qobj(operator[:dimension, :dimension])
    
diag_dressed_hamiltonian_trunc = truncate(diag_dressed_hamiltonian, total_truncation)
evalues = (diag_dressed_hamiltonian_trunc.eigenenergies() - diag_dressed_hamiltonian_trunc.eigenenergies()[0]) / 6.28
    
    # Compute chi
e_11 = evalues[hilbertspace.dressed_index((1,1))]
e_10 = evalues[hilbertspace.dressed_index((1,0))]
e_01 = evalues[hilbertspace.dressed_index((0,1))]
e_00 = evalues[hilbertspace.dressed_index((0,0))]
    
chi = e_11 - e_10 - e_01 + e_00



print(f"{chi*1000:.4f} Chi (MHz)")

print(f"{e_11:.4f}, {e_10:.4f}, {e_01:.4f}, {e_00:.4f} Energies (GHz) for (11, 10, 01, 00) states")
detuning = e_10 - e_01
# print(f"{detuning:.4f} Detuning (GHz)")


Chi = abs(chi)

-2.6408 Chi (MHz)
11.4894, 4.6805, 6.8116, 0.0000 Energies (GHz) for (11, 10, 01, 00) states


In [ ]:
# for Hanzhe's device parameters 15/08/2025
import scqubits as scq
import qutip as qt
import numpy as np


w_r = 6.75e9
w_ls = 3.0e9
w_rs = 5.5e9
wq = 5.3e9

EC = 0.16
EJ = 22.5 # 0.22 * 110
E_osc = w_r /1e9  # GHz, resonator frequency

tmon = scq.Transmon(
    EJ=EJ,      # Josephson energy
    EC=EC,      # Charging energy
    ng=0,     # Offset charge
    ncut=41,    # Charge basis cutoff
    truncated_dim=10  # Number of levels to keep
)


resonator = scq.Oscillator(E_osc=E_osc, truncated_dim=20)

hilbertspace = scq.HilbertSpace([tmon, resonator])

g = -0.04450*1j #g = 120 MHz, calculate this from  capacitance matrix

hilbertspace.add_interaction(
    expr="g * n * a - g * n * adag",
    op1=("n", tmon.n_operator),
    op2=("adag", resonator.creation_operator),
    op3 = ("a", resonator.annihilation_operator),
    add_hc=False
)

hilbertspace.generate_lookup()
    
    # Extract eigenvalues
evals = hilbertspace["evals"][0]
diag_dressed_hamiltonian = (
    2 * np.pi * qt.Qobj(np.diag(evals), dims=[hilbertspace.subsystem_dims] * 2)
    )

total_truncation = 10
def truncate(operator: qt.Qobj, dimension: int) -> qt.Qobj:
        return qt.Qobj(operator[:dimension, :dimension])
    
diag_dressed_hamiltonian_trunc = truncate(diag_dressed_hamiltonian, total_truncation)
evalues = (diag_dressed_hamiltonian_trunc.eigenenergies() - diag_dressed_hamiltonian_trunc.eigenenergies()[0]) / 6.28
    
    # Compute chi
e_11 = evalues[hilbertspace.dressed_index((1,1))]
e_10 = evalues[hilbertspace.dressed_index((1,0))]
e_01 = evalues[hilbertspace.dressed_index((0,1))]
e_00 = evalues[hilbertspace.dressed_index((0,0))]
    
chi = e_11 - e_10 - e_01 + e_00



print(f"{chi*1000:.4f} Chi (MHz)")

print(f"{e_11:.4f}, {e_10:.4f}, {e_01:.4f}, {e_00:.4f} Energies (GHz) for (11, 10, 01, 00) states")
detuning = e_10 - e_01
# print(f"{detuning:.4f} Detuning (GHz)")


Chi = abs(chi)

-0.6611 Chi (MHz)
11.9561, 5.2011, 6.7557, 0.0000 Energies (GHz) for (11, 10, 01, 00) states


In [4]:
# ...existing code...
import scqubits as scq
import qutip as qt
import numpy as np

# device / transmon / resonator params (GHz)
wq_GHz = 5.3
wr_GHz = 6.75
w_ls_GHz = 3.0
w_rs_GHz = 5.5

EC = 0.16
EJ = 22.5

# --- Use g values directly (MHz) - edit these as needed ---
g_qr_MHz  = 45   # example: 120 MHz coupling to resonator1
g_qls_MHz = 22.5  # example: 40 MHz coupling to resonator2 (unused below)
g_qrs_MHz = 12.5    # example: 30 MHz coupling to resonator3 (unused below)

# convert to GHz for scqubits
g_qr_GHz = g_qr_MHz / 1e3

# build systems
tmon = scq.Transmon(EJ=EJ, EC=EC, ng=0, ncut=41, truncated_dim=10)
resonator = scq.Oscillator(E_osc=wr_GHz, truncated_dim=20)
hs = scq.HilbertSpace([tmon, resonator])

# add interaction using g directly (scqubits expects g in GHz; use complex pref if desired)
g_pref = 1j * g_qr_GHz
hs.add_interaction(
    expr="g * n * a - g * n * adag",
    op1=("n", tmon.n_operator),
    op2=("adag", resonator.creation_operator),
    op3=("a", resonator.annihilation_operator),
    add_hc=False,
)
hs.generate_lookup()

# exact (numerical) chi via diagonalization
evals = hs["evals"][0]
diag_dressed_hamiltonian = 2 * np.pi * qt.Qobj(np.diag(evals), dims=[hs.subsystem_dims] * 2)

def truncate(operator: qt.Qobj, dimension: int) -> qt.Qobj:
    return qt.Qobj(operator[:dimension, :dimension])

diag_trunc = truncate(diag_dressed_hamiltonian, 10)
evalues_GHz = (diag_trunc.eigenenergies() - diag_trunc.eigenenergies()[0]) / (2 * np.pi)  # GHz

e_11 = evalues_GHz[hs.dressed_index((1,1))]
e_10 = evalues_GHz[hs.dressed_index((1,0))]
e_01 = evalues_GHz[hs.dressed_index((0,1))]
e_00 = evalues_GHz[hs.dressed_index((0,0))]
chi_exact_GHz = e_11 - e_10 - e_01 + e_00

print(f"Exact (diagonalization) chi: {chi_exact_GHz*1e3:.4f} MHz  ({chi_exact_GHz:.6e} GHz)")
print(f"Energies (GHz) e11,e10,e01,e00 = {e_11:.6f}, {e_10:.6f}, {e_01:.6f}, {e_00:.6f}")

# perturbative dispersive chi (transmon formula)
Delta_GHz = wq_GHz - wr_GHz              # Δ = ω_q - ω_r (GHz)
alpha_GHz = -EC                          # transmon anharmonicity ≈ -EC (GHz)
chi_pert_GHz = 2 * (g_qr_GHz**2) * alpha_GHz / (Delta_GHz * (Delta_GHz + alpha_GHz))

print(f"\nPerturbative chi: {chi_pert_GHz*1e3:.4f} MHz  ({chi_pert_GHz:.6e} GHz)")
print(f"Detuning Δ = {Delta_GHz:.6f} GHz, alpha = {alpha_GHz:.6f} GHz")

rel_diff = (chi_exact_GHz - chi_pert_GHz) / (chi_exact_GHz if chi_exact_GHz != 0 else 1.0)
print(f"\nRelative difference (perturbative vs exact): {rel_diff:.3f}")
# ...existing code...

Exact (diagonalization) chi: -0.6608 MHz  (-6.607839e-04 GHz)
Energies (GHz) e11,e10,e01,e00 = 11.950051, 5.198455, 6.752257, 0.000000

Perturbative chi: -0.2776 MHz  (-2.775755e-04 GHz)
Detuning Δ = -1.450000 GHz, alpha = -0.160000 GHz

Relative difference (perturbative vs exact): 0.580


In [2]:
# ...existing code...
import scqubits as scq
import qutip as qt
import numpy as np

# device / transmon / resonator params (GHz)
wq_GHz = 5.3
wr_GHz = 6.75
w_ls_GHz = 3.0
w_rs_GHz = 5.5

EC = 0.16
EJ = 22.5

# --- Use g values directly (MHz) - edit these as needed ---
g_qr_MHz  = 45.0
g_qls_MHz = 22.5
g_qrs_MHz = 12.5

def compute_chi_for_resonator(wr_here_GHz, g_MHz, label):
    g_GHz = g_MHz / 1e3
    # build systems
    tmon = scq.Transmon(EJ=EJ, EC=EC, ng=0, ncut=41, truncated_dim=10)
    resonator = scq.Oscillator(E_osc=wr_here_GHz, truncated_dim=20)
    hs = scq.HilbertSpace([tmon, resonator])

    # add interaction using g directly (scqubits expects g in GHz)
    g_pref = 1j * g_GHz
    hs.add_interaction(
        expr="g * n * a - g * n * adag",
        op1=("n", tmon.n_operator),
        op2=("adag", resonator.creation_operator),
        op3=("a", resonator.annihilation_operator),
        add_hc=False,
    )
    hs.generate_lookup()

    # exact (numerical) chi via diagonalization
    evals = hs["evals"][0]
    diag_dressed_hamiltonian = 2 * np.pi * qt.Qobj(np.diag(evals), dims=[hs.subsystem_dims] * 2)

    def truncate(operator: qt.Qobj, dimension: int) -> qt.Qobj:
        return qt.Qobj(operator[:dimension, :dimension])

    diag_trunc = truncate(diag_dressed_hamiltonian, 10)
    evalues_GHz = (diag_trunc.eigenenergies() - diag_trunc.eigenenergies()[0]) / (2 * np.pi)  # GHz

    e_11 = evalues_GHz[hs.dressed_index((1,1))]
    e_10 = evalues_GHz[hs.dressed_index((1,0))]
    e_01 = evalues_GHz[hs.dressed_index((0,1))]
    e_00 = evalues_GHz[hs.dressed_index((0,0))]
    chi_exact_GHz = e_11 - e_10 - e_01 + e_00

    # perturbative dispersive chi (transmon formula)
    Delta_GHz = wq_GHz - wr_here_GHz
    alpha_GHz = -EC
    chi_pert_GHz = 2 * (g_GHz**2) * alpha_GHz / (Delta_GHz * (Delta_GHz + alpha_GHz))

    print(f"\n{label}: g = {g_MHz:.3f} MHz | Exact chi = {chi_exact_GHz*1e3:.4f} MHz | Pert chi = {chi_pert_GHz*1e3:.4f} MHz")
    print(f"  Energies (GHz) e11,e10,e01,e00 = {e_11:.6f}, {e_10:.6f}, {e_01:.6f}, {e_00:.6f}")
    print(f"  Δ = {Delta_GHz:.6f} GHz, α = {alpha_GHz:.6f} GHz")
    return chi_exact_GHz, chi_pert_GHz

# compute for all three resonators
chi_qr_exact, chi_qr_pert = compute_chi_for_resonator(wr_GHz, g_qr_MHz, "Resonator 1 (wr)")
chi_qls_exact, chi_qls_pert = compute_chi_for_resonator(w_ls_GHz, g_qls_MHz, "Resonator 2 (w_ls)")
chi_qrs_exact, chi_qrs_pert = compute_chi_for_resonator(w_rs_GHz, g_qrs_MHz, "Resonator 3 (w_rs)")

# relative differences
def rel_diff(exact, pert):
    return (exact - pert) / (exact if exact != 0 else 1.0)

print(f"\nRelative diffs (pert vs exact): Res1={rel_diff(chi_qr_exact,chi_qr_pert):.3f}, Res2={rel_diff(chi_qls_exact,chi_qls_pert):.3f}, Res3={rel_diff(chi_qrs_exact,chi_qrs_pert):.3f}")
# ...existing code...
# filepath: /Users/tanvirahmedmasum/Documents/Coding/Circuit Quantization/Transmon chi calc from g value.ipynb
# ...existing code...
import scqubits as scq
import qutip as qt
import numpy as np

# device / transmon / resonator params (GHz)
wq_GHz = 5.3
wr_GHz = 6.75
w_ls_GHz = 3.0
w_rs_GHz = 5.5

EC = 0.16
EJ = 22.5

# --- Use g values directly (MHz) - edit these as needed ---
g_qr_MHz  = 45.0
g_qls_MHz = 22.5
g_qrs_MHz = 12.5

def compute_chi_for_resonator(wr_here_GHz, g_MHz, label):
    g_GHz = g_MHz / 1e3
    # build systems
    tmon = scq.Transmon(EJ=EJ, EC=EC, ng=0, ncut=41, truncated_dim=10)
    resonator = scq.Oscillator(E_osc=wr_here_GHz, truncated_dim=20)
    hs = scq.HilbertSpace([tmon, resonator])

    # add interaction using g directly (scqubits expects g in GHz)
    g = 1j * g_GHz
    hs.add_interaction(
        expr="g * n * a - g * n * adag",
        op1=("n", tmon.n_operator),
        op2=("adag", resonator.creation_operator),
        op3=("a", resonator.annihilation_operator),
        add_hc=False,
    )
    hs.generate_lookup()

    # exact (numerical) chi via diagonalization
    evals = hs["evals"][0]
    diag_dressed_hamiltonian = 2 * np.pi * qt.Qobj(np.diag(evals), dims=[hs.subsystem_dims] * 2)

    def truncate(operator: qt.Qobj, dimension: int) -> qt.Qobj:
        return qt.Qobj(operator[:dimension, :dimension])

    diag_trunc = truncate(diag_dressed_hamiltonian, 10)
    evalues_GHz = (diag_trunc.eigenenergies() - diag_trunc.eigenenergies()[0]) / (2 * np.pi)  # GHz

    e_11 = evalues_GHz[hs.dressed_index((1,1))]
    e_10 = evalues_GHz[hs.dressed_index((1,0))]
    e_01 = evalues_GHz[hs.dressed_index((0,1))]
    e_00 = evalues_GHz[hs.dressed_index((0,0))]
    chi_exact_GHz = e_11 - e_10 - e_01 + e_00

    # perturbative dispersive chi (transmon formula)
    Delta_GHz = wq_GHz - wr_here_GHz
    alpha_GHz = -EC
    chi_pert_GHz = 2 * (g_GHz**2) * alpha_GHz / (Delta_GHz * (Delta_GHz + alpha_GHz))

    print(f"\n{label}: g = {g_MHz:.3f} MHz | Exact chi = {chi_exact_GHz*1e3:.4f} MHz | Pert chi = {chi_pert_GHz*1e3:.4f} MHz")
    print(f"  Energies (GHz) e11,e10,e01,e00 = {e_11:.6f}, {e_10:.6f}, {e_01:.6f}, {e_00:.6f}")
    print(f"  Δ = {Delta_GHz:.6f} GHz, α = {alpha_GHz:.6f} GHz")
    return chi_exact_GHz, chi_pert_GHz

# compute for all three resonators
chi_qr_exact, chi_qr_pert = compute_chi_for_resonator(wr_GHz, g_qr_MHz, "Resonator 1 (wr)")
chi_qls_exact, chi_qls_pert = compute_chi_for_resonator(w_ls_GHz, g_qls_MHz, "Resonator 2 (w_ls)")
chi_qrs_exact, chi_qrs_pert = compute_chi_for_resonator(w_rs_GHz, g_qrs_MHz, "Resonator 3 (w_rs)")

# relative differences
def rel_diff(exact, pert):
    return (exact - pert) / (exact if exact != 0 else 1.0)

print(f"\nRelative diffs (pert vs exact): Res1={rel_diff(chi_qr_exact,chi_qr_pert):.3f}, Res2={rel_diff(chi_qls_exact,chi_qls_pert):.3f}, Res3={rel_diff(chi_qrs_exact,chi_qrs_pert):.3f}")

ValueError: Invalid Interaction Term. Exception: name 'g' is not defined